Based on:
https://www.pyimagesearch.com/2017/04/24/eye-blink-detection-opencv-python-dlib/

In [12]:
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2

from time import time
import matplotlib.pyplot as plt #dev

In [15]:
modelPath = 'models/shape_predictor_68_face_landmarks.dat'
imgPath = 'data/sample_img_2.jpg'
#'assets/blink_3.jpg'

In [7]:
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear

# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3

In [8]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(modelPath)

# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

[INFO] loading facial landmark predictor...


In [17]:
frame = cv2.imread(imgPath)
frame = imutils.resize(frame, width=450)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# detect faces in the grayscale frame
rects = detector(gray, 0)
if len(rects)>0:
    time0 = time()
    shape = predictor(gray, rects[0])
    shape = face_utils.shape_to_np(shape)
    time1 = time()
    print("Prediction time: %.2f ms" % ((time1 - time0)*1000))
    
    # extract the left and right eye coordinates, then use the
    # coordinates to compute the eye aspect ratio for both eyes
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)

    # average the eye aspect ratio together for both eyes
    ear = (leftEAR + rightEAR) / 2.0

    # compute the convex hull for the left and right eye, then
    # visualize each of the eyes
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
    cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

    # check to see if the eye aspect ratio is below the blink
    # threshold, and if so, increment the blink frame counter
    print(ear)
    if ear < EYE_AR_THRESH:
        pass
#Image 1: 0.12090964114213615
#Image 2: 0.23488808780588138
#Image 3: 0.388472372991739
#higher=more open

Prediction time: 2.35 ms
0.2479189881958119


# Possibilities
* Blink pattern over time
* Drowsiness detection (https://www.pyimagesearch.com/2017/05/08/drowsiness-detection-opencv/)
* Head pose estimation (https://www.learnopencv.com/head-pose-estimation-using-opencv-and-dlib/)
* much more